# Module 4 - Lab 1: Building a Regression Model with Spark

This lab will build upon the previous lab. 
While classification models deal with outcomes that represent discrete classes, regression models are concerned with target variables that can take any real value. Our goal is to find a model that maps input features to predicted target variables. 
Like classification, regression is also a form of supervised learning.

We can use regression models to predict a mulititude of different variables of interest. 
Here are some examples of how this technique is used in society:
* Predicting stock returns and other economic variables
* Predicting loss amounts for loan defaults (this can be combined with a classification model that predicts the probability of default, while the regression model predicts the amount in the case of a default)
* Predicting customer lifetime value (CLTV) in a retail, mobile, or other business, based on user behavior and spending patterns

Below, we will fire up our Spark cluster and prepare it for data visualization like we have done in previous labs. See Module 3 - Lab 1: Feature-Extraction-MoreContext for a detailed description of what is happening below. 

In [ ]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.tree import DecisionTree

# Make Random String
import os, random, string
length = 32
chars = string.ascii_letters + string.digits + '_-'
random.seed = (os.urandom(1024))

rndfolder = ''.join(random.choice(chars) for i in range(length))
dirpath = '/home/hadoop/work_dir/' + rndfolder + '/'

# Set Path and permissions ("0770", which means everyone can read and write the file)
os.mkdir(dirpath, 0770)
os.chdir(dirpath)

def process_figure(fig, name):
    fig.savefig(name)
    print 'http://ec2-54-153-99-19.us-west-1.compute.amazonaws.com:8810/' + rndfolder + '/' + name

import matplotlib
matplotlib.use('agg') # non-graphical mode (pngs(?))
import matplotlib.pyplot as plt

import numpy as np

print("Ready")

## Types of regression models
Let's talk about the different types of regression models before we dive into coding up an example.
Spark's MLlib library offers two broad classes of regression models: linear models and decision tree regression models.
Linear regression models use a different loss function, related link function, and decision function than its classification counterparts.
(We will go over these functions in minute.)
MLlib provides a standard least squares regression model, which is a type of a linear model.

### Least square regression
There is a variety of loss functions that can be applied to generalized linear models. 
(A loss function is a function that maps an event (or values of one or more variables) onto a real number intuitively representing some "cost" associated with the event.)

This is the loss function used for least squares (a.k.a. the squared loss):
    1/2[(w^T) * x - y]^2
* y: the target variable (it has a real value)
* w: weight vector
* x: feature vector
* T: instance? 

So what is this useful for? 
What does all of this actually mean?
Least Squares Regression is the method for summarizing a pattern in a dataset that suggests a linear relationship.
This allows us to predict an outcome, y, for a given input, x. 
(i.e. We can describe (or predict) how a response variable, y, changes as an explanatory variable, x, changes.)
This model is useful for SPECIFIC situations.
The standard least squares regression in MLlib does not use regularization (which is a process of introducing additional information in order to solve an ill-posed problem or to prevent overfitting).
We can see that the loss applied to incorrectly predicted points will be magnified since the loss is squared.
This means that least squares regression is susceptible to outliers in the dataset and also to over-fitting.
So, generally, we should apply some level of regularization to account for this.

Here are some things that may happen using this technique against the data (so if you aren't sure what the data looks like and you use this method, the following situations could occur):
* A curved pattern might appear showing that the relationship is not linear
* Increasing or decreasing spread about the line as x increases indicates that prediction of y will be LESS accurate for larger x's.
* Individual points with large residuals are outliers  in the vertical direction
* Individual points that are extreme in the x direction are also important....as influential observations
This link is helpful for explaining this concept even further: http://www.henry.k12.ga.us/ugh/apstat/chapternotes/sec3.3.html
(I recommend reading it. It's not too long, and I think it is helpful to explain the math behind the method.)

The link function is the identity link.
The decision function is also the identity function, since generally, no thresholding is applied in regression. 
Therefore, the model's prediction is simply: y = (w^T) * x

Linear regression with L2 regularization is commonly referred to as ridge regression, while applying L1 regularization is called the lasso.
To understand this concept better, read the "Regularization" section of this website: http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/

### Decision trees for regression
The Decision tree method builds regression or classification models in the form of a tree structure. 
It brakes down a dataset into smaller and smaller subsets while incrementally developing an associated decision tree at the same time.
The final result is a tree with decision nodes and leaf nodes.

* A decision node has two or more branches each representing values for the attribute being tested.
* A leaf node represents a decision on the numerical target. 
* The topmost node is called the "root node".

Decision trees can handle both categorical and numerical data.
A decision tree is built top-down from a root node and involves partitioning the data into subsets that contain instances with similar values (homogenous).
We use standard deviation to calculate the homogeneity of a numerical sample.
If the numerical sample is completely homogeneous its standard deviation is zero.
The standard deviation reduction is based on the decrease in standard deviation after a dataset is split on an attribute. 
Constructing a decision tree is all about finding attribute that returns the highest standard deviation reduction (i.e., the most homogeneous branches).
I recommend reading this content for a more detailed explanation of this concept: 
http://chem-eng.utoronto.ca/~datamining/dmc/decision_tree_reg.htm

Just like using linear models for regression tasks involves changing the loss function used, using decision trees for regression involves changing the measure of the node impurity used.
The impurity metric is called variance and is defined in the same way as the squared loss for least squares linear regression.

## Extracting the right features from you data
As the underlying models for regression are the same as those for the classification case, we can use the same approach to create input features. 
The only practical difference is that the target is now a real-valued variable, as opposed to a categorical one. 
The `LabeledPoint` class in MLlib already takes this into account, as the label field is of the `Double` type, so it can handle both cases.

### Extracting features from the bike sharing dataset
To illustrate the concepts in this chapter, we will be using the bike sharing dataset. 
This dataset contains hourly records of the number of bicycle rentals in the capital bike sharing system. 
It also contains variables related to date and time, weather, and seasonal and holiday information.

Here are the variable names and descriptions:

* `instant`: This is the record ID
* `dteday`: This is the raw data
* `season`: This is different seasons such as spring, summer, winter, and fall
* `yr`: This is the year (2011 or 2012)
* `mnth`: This is the month of the year
* `hr`: This is the hour of the day
* `holiday`: This is whether the day was a holiday or not
* `weekday`: This is the day of the week
* `workingday`: This is whether the day was a working day or not
* `weathersit`: This is a categorical variable that describes the weather at a particular time
* `temp`: This is the normalized temperature
* `atemp`: This is the normalized apparent temperature
* `hum`: This is the normalized humidity
* `windspeed`: This is the normalized wind speed
* `cnt`: This is the target variable, that is, the count of bike rentals for that hour


In [ ]:
path = "bike/hour_noheader.csv"
raw_data = sc.textFile(path)
num_data = raw_data.count()
records = raw_data.map(lambda x: x.split(","))
first = records.first()
print first
print num_data

This code above loads the "bike/hour_noheader.csv" dataset into the Spark instance that is created (similar to previous datasets that we have used in earlier modules).
The data is then counted in the line `num_data = raw_data.count()`. 
The data is then parsed in this line `records = raw_data.map(lambda x: x.split(","))`. 

* The lambda expression splits each line in the data set by the comma (,) character and returns the list to be stored in the `records` variable.

The first line in the records dataset is selected in this line `first = records.first()`.
It is then printed and along with the number of records that was previously counted. 
As we can see from the results, we have 17,379 hourly records in our dataset. 
For now, we will ignore the record ID and raw date columns.
We will also ignore the `casual` and `registered` count target variables and focus on the overall count variable, `cnt` (which is the sum of the other two counts).
We are left with 12 variables. 
The first eight are categorical, while the last 4 are normalized real-valued variables.

To deal with the eight categorical variables, we will use the binary encoding approach.
The four real-valued variables will be left as is.

First, we will cache the dataset since we will be reading from it a lot in this lab.

In [ ]:
records.cache()

In order to extract each categorical feature into a binary vector form, we will need to know the feature mapping of each feature value to the index of the nonzero value in our binary vector.
So this means....what?

* We need to use a binary vector to store the codes for the eight categorical variabes the we get from our binary encoding method. (We haven't done this yet.)
* To perform this method, we have to know how each feature maps to an index within the binary vector. 

Let's define a function that will extract this mapping from our dataset for a given column.
The function below will do this.

In [ ]:
def get_mapping(rdd, idx):
    return rdd.map(lambda fields: fields[idx]).distinct().zipWithIndex().collectAsMap()

The function `get_mapping(rdd, idx)` first maps the field to its unique values. 
Then, it uses the `zipWithIndex()` transformation to "zip" the value up with a unique index such that a key-value RDD is formed.
In the formulated RDD, the key is the variable and the value is the index.

* This index will be the index of the nonzero entry in the binary vector representation of the feature.

We then collect this RDD back to the driver as a Python dictionary via the `collectAsMap()` function.

Let's test our function.

In [ ]:
print "Mapping of first categorical feature column: %s" % get_mapping(records, 2)

Now, let's apply the function to each categorical column (i.e. for variable indices 2 to 9).

In [ ]:
mappings = [get_mapping(records, i) for i in range(2,10)]
cat_len = sum(map(len, mappings))
num_len = len(records.first()[11:15])
total_len = num_len + cat_len

We now have the mappings for each variable (as seen in the code above), and we can see how many values in total we need for our binary vector representation in the code below.

In the code block above:

* `mappings = [get_mapping(records, i) for i in range(2,10)]` obtains the mappings that we desire for each variable.
* `cat_len = sum(map(len, mappings))` obtains the length of each variable's mapping, and then sums them.
* `num_len = len(records.first()[11:15])` obtains the length of the values in the 11th-15th indices in the first list in records
    * `first()` gets the first item from the `records` list
* `total_len = num_len + cat_len` calculates the total length of the values, which will be necessary for our tree construction

In [ ]:
print "Feature vector length for categorical features: %d" % cat_len
print "Feature vector length for numerical features: %d" % num_len
print "Total feature vector length: %d" % total_len

# Creating feature vectors for linear model
The next step is to use our extracted mappings to convert the categorical features to binary-encoded features. 
Again, it will be helpful to create a function that we can apply to each record in our dataset for this purpose. 
We will also create a function to extract the target variable from each record.

In [ ]:
def extract_features(record):
    cat_vec = np.zeros(cat_len)
    i = 0
    step = 0
    for field in record[2:9]:
        m = mappings[i]
        idx = m[field]
        cat_vec[idx + step] = 1
        i = i + 1
        step = step + len(m)
    num_vec = np.array([float(field) for field in record[10:14]])
    return np.concatenate((cat_vec, num_vec))
def extract_label(record):
    return float(record[-1])

In the function `extract_features(record)`, we ran through each column in the row of data via the `for` loop.
Before the `for` loop, we created a vector of zeros in the line `cat_vec = np.zeros(cat_len)`.
We extracted the binary encoding for each variable in turn from the mappings we created previously via the line `m = mappings[i]`.
We then assign the binary encoding to the `idx` variable in line `idx = m[field]`. 
In the line `cat_vec[idx + step] = 1` we encode a `1` at the location `idx + step` in the `cat_vec` vector.
We increment the value of `i` to move forward in the mappings vector. 
The `step` variable ensures that the nonzero feature index in the full feature vector (a.k.a. `cat_vec`) is correct (and is somewhat more efficient than, say, creating many smaller binary vectors and concatenating them).
The numeric vector (`num_vec`) is created directly by first converting the data to floating point numbers and wrapping these in a numpy array. 
The resulting two vectors are then concatenated and returned.

The `extract_label` function simply converts the last column variable (the count) into a float.

In [ ]:
data = records.map(lambda r: LabeledPoint(extract_label(r), extract_features(r)))

Above, we extract labels and feature vectors from our records data using the `extract_label()` and `extract_features()` functions.
The `LabeldPoint(label, feature)` function takes in a label (denoted `label`) and a feature (denoted `feature`), which is a vector of features for this point.
The class `LabeledPoint` represents the features and labels of a data point.

Let's now inspect the first record in the extracted feature RDD (a.k.a. the `data` RDD that we created above).

In [ ]:
first_point = data.first()
print "Raw data: " + str(first[2:])
print "Label: " + str(first_point.label)
print "Linear Model feature vector:\n" + str(first_point.features)
print "Linear Model feature vector length: " + str(len(first_point.features))

As we can see, we converted the raw data into a feature vector made up of the binary categorical and real numeric features, and we indeed have a total vector length of 61.

So what does this tell us?
The Linear Model feature vector shown above shows the binary encoding for each categorical feature that was present before in the raw data. 
We will now move on, so that we can use this feature vector in more useful ways.
(Obviously, just looking at the values of a giant vector is not as useful as looking at a visual representation or doing performance metrics on the vector itself.)

# Creating feature vectors for the decision tree
Now, we will create a separate function to extract the decision tree feature vector, which simply converts all the values to floats and wraps them in a numpy array.

In [ ]:
def extract_features_dt(record):
    return np.array(map(float, record[2:14]))
data_dt = records.map(lambda r: LabeledPoint(extract_label(r), extract_features_dt(r)))
first_point_dt = data_dt.first()
print "Decision Tree feature vector: " + str(first_point_dt.features)
print "Decision Tree feature vector length: " + str(len(first_point_dt.features))

# Training and using regression models
Training for regression models using decision trees and linear models follows the same procedure as for classi cation models. We simply pass the training data contained in a [LabeledPoint] RDD to the relevant `train` method. 
In Python, we are provided with a convenience method that gives us access to all the available model arguments, so we only have to use this one entry point for training. 
We can see the details of these convenience functions by calling the `help()` function on the `train` methods.

In [ ]:
help(LinearRegressionWithSGD.train)

In [ ]:
help(DecisionTree.trainRegressor)

## Training a regression model on the bike sharing dataset
We're ready to use the features we have extracted to train our models on the bike sharing data. 
First, we'll train the linear regression model and take a look at the first few predictions that the model makes on the data.

In [ ]:
linear_model = LinearRegressionWithSGD.train(data, iterations=10, step=0.1, intercept=False)
true_vs_predicted = data.map(lambda p: (p.label, linear_model.predict(p.features)))
print "Linear Model predictions: " + str(true_vs_predicted.take(5))

The above code block does the following:

`linear_model = LinearRegressionWithSGD.train(data, iterations=10, step=0.1, intercept=False)`

* The function `LinearRegressionWithSGD.train()` provides our linear model. (The details can be read above.)

`true_vs_predicted = data.map(lambda p: (p.label, linear_model.predict(p.features)))`

* Here, we obtain the "label", which is the value we expect from the data, and the prediction (what we expect based on our model).

Note that we have not used the default settings for `iterations` and `step` here. 
We've changed the number of iterations so that the model does not take too long to train.
With more iterations, we get a better model - one that can better predict output when given a set of intput.
However, it also takes more time to train the model. 

Next, we will train the decision tree model simply using the default arguments to the `trainRegressor` method (which equates to using a tree depth of 5). 
Note that we need to pass in the other form of the dataset, `data_dt`, that we created from the raw feature values (as opposed to the binary encoded features that we used for the preceding linear model).

We also need to pass in an argument for `categoricalFeaturesInfo`. 
This is a dictionary that maps the categorical feature index to the number of categories for the feature. 
If a feature is not in this mapping, it will be treated as continuous. 
For our purposes, we will leave this as is, passing in an empty mapping. 

In [ ]:
dt_model = DecisionTree.trainRegressor(data_dt,{})
preds = dt_model.predict(data_dt.map(lambda p: p.features))
actual = data.map(lambda p: p.label)
true_vs_predicted_dt = actual.zip(preds)
print "Decision Tree predictions: " + str(true_vs_predicted_dt.take(5))
print "Decision Tree depth: " + str(dt_model.depth())
print "Decision Tree number of nodes: " + str(dt_model.numNodes())

After observing the output of both of these methods, it appears that the decision tree may do better. 
(The linear modeal is WAY off in all of its predictions.)
However, we will perform more stringent evaluation methods to find out.

# Evaluating performance
When dealing with regression models, it is very unlikely that our model will precisely predict the target variable, because the target variable can take on any real value. 
However, we would naturally like to understand how far away our predicted values are from the true values, so will we utilize a metric that takes into account the overall deviation.

Some of the standard evaluation metrics used to measure the performance of regression models include the Mean Squared Error (MSE) and Root Mean Squared Error (RMSE), the Mean Absolute Error (MAE), the R-squared coefficient, and many others.

## Mean Squared Error and Root Mean Squared Error
MSE is the average of the squared error that is used as the loss function for least squares regression:

I will attempt to write the equation here, but no guarantees it will look nice.. 

Summation from i = 1 to n of [ (w^T * x(i) - y(i))^2 / n ]

It is the sum, over all the data points, of the square of the difference between the predicted and actual target variables, divided by the number of data points.
(Target variables are our predicted values.)

RMSE is the square root of MSE. 
MSE is measured in units that are the square of the target variable, while RMSE is measured in the same units as the target variable. 
Due to its formulation, MSE, just like the squared loss function that it derives from, effectively penalizes larger errors more severely.

In order to evaluate our predictions based on the mean of an error metric, we will first make predictions for each input feature vector in an RDD of `LabeledPoint` instances by computing the error for each record using a function that takes the prediction and true target value as inputs. 
This will return a [`Double`] RDD (in other words, an RDD that contains items of type `Double`) that contains the error values. 
We can then find the average using the `mean` method of RDDs that contain `Double` values.

Below, we define our squared error function.

In [ ]:
def squared_error(actual, pred):
    return (pred - actual)**2

The above function takes in two variables, `actual` and `pred`, which represent the actual value and the predicted value, respectively. 
It then returns the the squared difference between `pred` and `actual`.

## Mean Absolute Error
MAE is the average of the absolute differences between the predicted and actual targets:

Again, I will attempt to describe the equation..

Summation from i = 1 to n of [ | w^T * x(i) - y(i) | / n ]

MAE is similar in principle to MSE, but it does not punish large deviations as much.

In [ ]:
def abs_error(actual, pred):
    return np.abs(pred - actual)

## Root Mean Squared Log Error
This measurement is not as widely used as MSE and MAE, but it is used as the metric for the Kaggle competition that uses the bike sharing dataset. 
It is effectively the RMSE of the log-transformed predicted and target values. 
This measurement is useful when there is a wide range in the target variable, and you do not necessarily want to penalize large errors when the predicted and target values are themselves high. 
It is also effective when you care about percentage errors rather than the absolute value of errors.

Below is the function to compute the RMSLE. 
CHALLENGE: See if you can write the mathematical equation! 

In [ ]:
def squared_log_error(pred, actual):
    return (np.log(pred + 1) - np.log(actual + 1))**2

## The R-squared coefficient
Here, we will briefly talk about the R-squared coefficients.

The R-squared coefficient, also known as the coefficient of determination, is a measure of how well a model fits a dataset. 
It is commonly used in statistics. 
It measures the degree of variation in the target variable (our predicted variable); this is explained by the variation in the input features. 
An R-squared coefficient generally takes a value between 0 and 1, where 1 equates to a perfect  fit of the model.

# Computing performance metrics on the bike sharing dataset
Given the functions we de ned earlier, we can now compute the various evaluation metrics on our bike sharing data.

## Linear model
Our approach will be to apply the relevant error function to each record in the RDD we computed earlier, which is `true_vs_predicted` for our linear model:


In [ ]:
mse = true_vs_predicted.map(lambda (t, p): squared_error(t, p)).mean()
mae = true_vs_predicted.map(lambda (t, p): abs_error(t, p)).mean()
rmsle = np.sqrt(true_vs_predicted.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Linear Model - Mean Squared Error: %2.4f" % mse
print "Linear Model - Mean Absolute Error: %2.4f" % mae
print "Linear Model - Root Mean Squared Log Error: %2.4f" % rmsle

In the code block above:

* We calculate the squared error of each tuple in the `true_vs_predicted` variable, and then we calculate the mean of that dictionary, which gives us the mean squared error.
* We then calculate the mean average error by calculating the absolute error on every tuple in the `true_vs_predicted` variable and then, taking the mean of that that dictionary.
* We then calculate the root mean squared log error. To do this, we have to take the mean of the dictionary created from applying the `squared_log_error` function to each tuple in the `true_vs_predicted` dictionary.

We then print the values.

# Decision tree
We will use the same approach for the decision tree model, using the `true_vs_ predicted_dt` RDD:

In [ ]:
mse_dt = true_vs_predicted_dt.map(lambda (t, p): squared_error(t, p)).mean()
mae_dt = true_vs_predicted_dt.map(lambda (t, p): abs_error(t, p)).mean()
rmsle_dt = np.sqrt(true_vs_predicted_dt.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Decision Tree - Mean Squared Error: %2.4f" % mse_dt
print "Decision Tree - Mean Absolute Error: %2.4f" % mae_dt
print "Decision Tree - Root Mean Squared Log Error: %2.4f" % rmsle_dt

Looking at the results, we can see that our initial guess about the decision tree model being the better performer is indeed true.

# Improving model performance and tuning parameters
It is true that feature transformation and selection can make a large difference to the performance of a model.
Here, we will focus on another type of transformation that can be applied to a dataset: transforming the target variable itself.
We will do this in a variety of ways and plot histograms to observe the changes. 

## Transforming the target variable
Recall that many machine learning models, including linear models, make assumptions regarding the distribution of the input data as well as target variables. 
In particular, linear regression assumes a normal distribution.

In many real-world cases, the distributional assumptions of linear regression do not hold. 
In this case, for example, we know that the number of bike rentals can never be negative. 
This alone should indicate that the assumption of normality might be problematic. 
To get a better idea of the target distribution, it is often a good idea to plot a histogram of the target values.

We will now create a plot of the target variable distribution in the following piece of code:

In [ ]:
targets = records.map(lambda r: float(r[-1])).collect()
plt.hist(targets, bins=40, color='lightblue', normed=True)
fig = plt.gcf()
fig.set_size_inches(16, 10)
process_figure(fig, 'target_distribution.png')
plt.clf()

Looking at the histogram plot, we can see that the distribution is highly skewed and certainly does not follow a normal distribution.

One way in which we might deal with this situation is by applying a transformation to the target variable, such that we take the logarithm of the target value instead of the raw value.
This is often referred to as log-transforming the target variable (this transformation can also be applied to feature values).

We will apply a log transformation to the following target variable and plot a histogram of the log-transformed values in the code block below.

In [ ]:
log_targets = records.map(lambda r: np.log(float(r[-1]))).collect()
plt.hist(log_targets, bins=40, color='lightblue', normed=True)
fig = plt.gcf()
process_figure(fig, 'log_target.png')
fig.set_size_inches(16, 10)
plt.clf()

We can now see that the data is more normally distributed. Notice that it is now skewed the other direction.

A second type of transformation that is useful in the case of target values that do not take on negative values and, in addition, might take on a very wide range of values, is to take the square root of the variable.

We will apply the square root transform in the following code, once more plotting the resulting target variable distribution in the code block below.

In [ ]:
sqrt_targets = records.map(lambda r: np.sqrt(float(r[-1]))).collect()
plt.hist(sqrt_targets, bins=40, color='lightblue', normed=True)
fig = plt.gcf()
fig.set_size_inches(16, 10)
process_figure(fig, 'sqrt_target.png')
plt.clf()

From the plots of the log and square root transformations, we can see that both
result in a more even distribution relative to the raw values. 
While they are still not normally distributed, they are a lot closer to a normal distribution when compared to the original target variable.

# Impact of training
So, does applying these transformations have any impact on model performance? 
Let's evaluate the various metrics we used previously on log-transformed data as an example.
We will do this first for the linear model by applying the numpy `log` function to the `label` field of each `LabeledPoint` RDD. 
Here, we will only transform the target variable, and we will not apply any transformations to the features.

In [ ]:
data_log = data.map(lambda lp: LabeledPoint(np.log(lp.label), lp.features))

We will then train a model on this transformed data and form the RDD of predicted versus true values.

In [ ]:
model_log = LinearRegressionWithSGD.train(data_log, iterations=10, step=0.1)

Note that now that we have transformed the target variable, the predictions of the model will be on the log scale, as will the target values of the transformed dataset. 
Therefore, in order to use our model and evaluate its performance, we must first transform the log data back into the original scale by taking the exponent of both the predicted and true values using the `numpy exp` function. 
We will do this in the code below.

In [ ]:
true_vs_predicted_log = data_log.map(lambda p: (np.exp(p.label), np.exp(model_log.predict(p.features))))

Now that our model is transformed back to its original state, we can calculate the MSE, MAE, and the RMSLE metrics for this model.
This is done below.

In [ ]:
mse_log = true_vs_predicted_log.map(lambda (t, p): squared_error(t, p)).mean()
mae_log = true_vs_predicted_log.map(lambda (t, p): abs_error(t, p)).mean()
rmsle_log = np.sqrt(true_vs_predicted_log.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Mean Squared Error: %2.4f" % mse_log
print "Mean Absolue Error: %2.4f" % mae_log
print "Root Mean Squared Log Error: %2.4f" % rmsle_log
print "Non log-transformed predictions:\n" + str(true_vs_predicted.take(3))
print "Log-transformed predictions:\n" + str(true_vs_predicted_log.take(3))

If we compare these results to the results on the raw target variable, we see that while we did not improve the MSE or MAE, we improved the RMSLE.

We will perform the same analysis for the decision tree model.

In [ ]:
data_dt_log = data_dt.map(lambda lp: LabeledPoint(np.log(lp.label), lp.features))
dt_model_log = DecisionTree.trainRegressor(data_dt_log,{})
preds_log = dt_model_log.predict(data_dt_log.map(lambda p: p.features))
actual_log = data_dt_log.map(lambda p: p.label)
true_vs_predicted_dt_log = actual_log.zip(preds_log).map(lambda (t, p): (np.exp(t), np.exp(p)))
mse_log_dt = true_vs_predicted_dt_log.map(lambda (t, p): squared_error(t, p)).mean()
mae_log_dt = true_vs_predicted_dt_log.map(lambda (t, p): abs_error(t, p)).mean()
rmsle_log_dt = np.sqrt(true_vs_predicted_dt_log.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Mean Squared Error: %2.4f" % mse_log_dt
print "Mean Absolue Error: %2.4f" % mae_log_dt
print "Root Mean Squared Log Error: %2.4f" % rmsle_log_dt
print "Non log-transformed predictions:\n" + str(true_vs_predicted_dt.take(3))
print "Log-transformed predictions:\n" + str(true_vs_predicted_dt_log.take(3))

From these results, we can see that we actually made our metrics slightly worse for the decision tree.

It is probably not surprising that the log transformation results in a better RMSLE performance for the linear model. 
As we are minimizing the squared error, once we have transformed the target variable to log values, we are effectively minimizing a loss function that is very similar to the RMSLE.

# Parameter Tuning
So far, we have illustrated the concepts of model training and evaluation for MLlib's regression models by training and testing on the same dataset.
We will now use a similar cross-validation approach that we used previously to evaluate the effect on performance of different parameter settings for our models.

## Creating training and testing sets to evaluate parameters
The first step is to create a test and training set for cross-validation purposes.
We will need to create a training and test dataset manually, since Spark's Python API does not yet provide the methods available in other languages.

One relatively easy way to do this is by first taking a random sample of, say, twenty percent of our data as our test set. 
We will then define our training set as the elements of the original RDD that are not in the test set RDD.

We can achieve this using the sample method to take a random sample for our test set, followed by using the `subtractByKey method`, which takes care of returning the elements in one RDD where the keys do not overlap with the other RDD.

Note that `subtractByKey`, as the name suggests, works on the keys of the RDD elements that consist of key-value pairs. 
Therefore, here we will use `zipWithIndex` on our RDD of extracted training examples. This creates an RDD of (`LabeledPoint`, `index`) pairs.

We will then reverse the keys and values so that we can operate on the index keys.

In [ ]:
data_with_idx = data.zipWithIndex().map(lambda (k, v): (v, k))
test = data_with_idx.sample(False, 0.2, 42)
train = data_with_idx.subtractByKey(test)

Once we have the two RDDs, we will recover just the `LabeledPoint` instances we need for training and test data, using map to extract the value from the key-value pairs.

In [ ]:
train_data = train.map(lambda (idx, p): p)
test_data = test.map(lambda (idx, p) : p)
train_size = train_data.count()
test_size = test_data.count()
print "Training data size: %d" % train_size
print "Test data size: %d" % test_size
print "Total data size: %d " % num_data
print "Train + Test size : %d" % (train_size + test_size)

The final step is to apply the same approach to the features extracted for the decision tree model.

In [ ]:
data_with_idx_dt = data_dt.zipWithIndex().map(lambda (k, v): (v, k))
test_dt = data_with_idx_dt.sample(False, 0.2, 42)
train_dt = data_with_idx_dt.subtractByKey(test_dt)
train_data_dt = train_dt.map(lambda (idx, p): p)
test_data_dt = test_dt.map(lambda (idx, p) : p)

## The impact of parameter settings for linear models
Now that we have prepared our training and test sets, we are ready to investigate the impact of different parameter settings on model performance. 
We will first carry out this evaluation for the linear model. 
We will create a convenience function to evaluate the relevant performance metric by training the model on the training set and evaluating it on the test set for different parameter settings.

We will use the RMSLE evaluation metric.
The evaluation function is defined below.

In [ ]:
def evaluate(train, test, iterations, step, regParam, regType, intercept):
    model = LinearRegressionWithSGD.train(train, iterations, step,
                                          regParam=regParam, regType=regType, intercept=intercept)
    tp = test.map(lambda p: (p.label, model.predict(p.features)))
    rmsle = np.sqrt(tp.map(lambda (t, p): squared_log_error(t, p)).mean())
    return rmsle

Note that in the following sections, you might get slightly different results due to some random initialization for SGD. 
However, your results will be comparable.

## Iterations
We generally expect that a model trained with SGD will achieve better performance as the number of iterations increases, although the increase in performance will slow down as the number of iterations goes above some minimum number. 
Note that here, we will set the step size to 0.01 to better illustrate the impact at higher iteration numbers.

The output shows that the error metric indeed decreases as the number of iterations increases.
It also does so at a decreasing rate, again as expected. 
What is interesting is that eventually, the SGD optimization tends to overshoot the optimal solution, and the RMSLE eventually starts to increase slightly.

In [ ]:
params = [1, 5, 10, 20, 50, 100]
metrics = [evaluate(train_data, test_data, param, 0.01, 0.0, 'l2', False) for param in params]
print params
print metrics

Here, we will use the `matplotlib` library to plot a graph of the RMSLE metric against the number of iterations. 
We will use a log scale for the x axis to make the output easier to visualize.

In [ ]:
plt.plot(params, metrics)
fig = plt.gcf()
plt.xscale('log')
process_figure(fig, 'iteration_stats.png')
plt.clf()

# Step size
We will perform a similar analysis for step size in the following code.

In [ ]:
params = [0.01, 0.025, 0.05, 0.1, 1.0]
metrics = [evaluate(train_data, test_data, 10, param, 0.0, 'l2', False) for param in params]
print params
print metrics

Now, we can see why we avoided using the default step size when training the linear model originally. 
The default is set to 1.0, which, in this case, results in a nan output for the RMSLE metric.
This typically means that the SGD model has converged to a very poor local minimum in the error function that it is optimizing. 
This can happen when the step size is relatively large, as it is easier for the optimization algorithm to overshoot good solutions.

We can also see that for low step sizes and a relatively low number of iterations (we used ten here), the model performance is slightly poorer. 
However, in the preceding **Iterations** section, we saw that for the lower step-size setting, a higher number of iterations will generally converge to a better solution.

Generally speaking, setting step size and number of iterations involves a trade-off. 
A lower step size means that convergence is slower but slightly more assured. 
However, it requires a higher number of iterations, which is more costly in terms of computation and time, in particular at a very large scale.

**NOTE**: Selecting the best parameter settings can be an intensive process that involves training a model on many combinations of parameter settings and selecting the best outcome.
Each instance of model training involves a number of iterations, so this process can be very expensive and time consuming when performed on very large datasets.

Below, we plot the output, again, using a log scale for the step-size axis.

In [ ]:
plt.plot(params, metrics)
fig = plt.gcf()
plt.xscale('log')
process_figure(fig, 'step_stats.png')
plt.clf()

# L2 regularization
Regularization has the effect of penalizing model complexity in the form of an additional loss term that is a function of the model weight vector. 
L2 regularization penalizes the L2-norm of the weight vector, while L1 regularization penalizes the L1-norm.

We expect training set performance to deteriorate with increasing regularization, as the model cannot fit the dataset well. 
However, we would also expect some amount of regularization that will result in optimal generalization performance as evidenced by the best performance on the test set.

Below, we will evaluate the impact of different levels of L2 regularization.

In [ ]:
params = [0.0, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0]
metrics = [evaluate(train_data, test_data, 10, 0.1, param, 'l2', False) for param in params]
print params
print metrics
plt.plot(params, metrics)
fig = plt.gcf()
plt.xscale('log')
process_figure(fig, 'l2_stats.png')
plt.clf()

As expected, there is an optimal setting of the regularization parameter with respect to the test set RMSLE.
This is easiest to see in the above plot (where we once more use the log scale for the regularization parameter axis).

# L1 Regularization
We can apply the same approach for differing levels of L1 regularization.

In [ ]:
params = [0.0, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
metrics = [evaluate(train_data, test_data, 10, 0.1, param, 'l1', False) for param in params]
print params
print metrics
plt.plot(params, metrics)
fig = plt.gcf()
plt.xscale('log')
process_figure(fig, 'l1_stats.png')
plt.clf()

Again, the results are more clearly seen when plotted in the following graph. 
We see that there is a much more subtle decline in RMSLE, and it takes a very high value to cause a jump back up. 
Here, the level of L1 regularization required is much higher than that for the L2 form; however, the overall performance is poorer.

Using L1 regularization can encourage sparse weight vectors. 
Does this hold true in this case? 
We can find out by examining the number of entries in the weight vector that are zero, with increasing levels of regularization.

We can see from the results that as we might expect, the number of zero feature weights in the model weight vector increases as greater levels of L1 regularization are applied.

In [ ]:
model_l1 = LinearRegressionWithSGD.train(train_data, 10, 0.1, regParam=1.0, regType='l1', intercept=False)
model_l1_10 = LinearRegressionWithSGD.train(train_data, 10, 0.1, regParam=10.0, regType='l1', intercept=False)
model_l1_100 = LinearRegressionWithSGD.train(train_data, 10, 0.1, regParam=100.0, regType='l1', intercept=False)
print "L1 (1.0) number of zero weights: " + str(sum(model_l1.weights.array == 0))
print "L1 (10.0) number of zeros weights: " + str(sum(model_l1_10.weights.array == 0))
print "L1 (100.0) number of zeros weights: " + str(sum(model_l1_100.weights.array == 0))

# Intercept
The final parameter option for the linear model is whether to use an intercept or not. 
An intercept is a constant term that is added to the weight vector and effectively accounts for the mean value of the target variable. 
If the data is already centered or normalized, an intercept is not necessary; however, it often does not hurt to use one in any case.
Below, we will evaluate the effect of adding an intercept term to the model.

In [ ]:
params = [False, True]
metrics = [evaluate(train_data, test_data, 10, 0.1, 1.0, 'l2', param) for param in params]
print params
print metrics
plt.bar(params, metrics, color='lightblue')
fig = plt.gcf()
process_figure(fig, 'intercept_stats.png')
plt.clf()

We can see from the results that as we might expect, the number of zero feature weights in the model weight vector increases as greater levels of L1 regularization are applied.

# The impact of parameter settings for the decision tree
Decision trees provide two main parameters: maximum tree depth and the maximum number of bins.
We will now perform the same evaluation of the effect of parameter settings for the decision tree model. 
Our starting point is to create an evaluation function for the model, similar to the one used for the linear regression earlier. 
This function is provided below.

In [ ]:
def evaluate_dt(train, test, maxDepth, maxBins):
    model = DecisionTree.trainRegressor(train, {}, impurity='variance', maxDepth=maxDepth, maxBins=maxBins)
    preds = model.predict(test.map(lambda p: p.features))
    actual = test.map(lambda p: p.label)
    tp = actual.zip(preds)
    rmsle = np.sqrt(tp.map(lambda (t, p): squared_log_error(t, p)).mean())
    return rmsle

# Tree Depth
We would generally expect performance to increase with more complex trees (that is, trees of greater depth). 
Having a lower tree depth acts as a form of regularization, and it might be the case that as with L2 or L1 regularization in linear models, there is a tree depth that is optimal with respect to the test set performance.

Here, we will try to increase the depths of trees to see what impact they have on test set RMSLE, keeping the number of bins at the default level of thirty-two.


In [ ]:
params = [1, 2, 3, 4, 5, 10, 20]
metrics = [evaluate_dt(train_data_dt, test_data_dt, param, 32) for param in params]
print params
print metrics
plt.plot(params, metrics)
fig = plt.gcf()
process_figure(fig, 'tree_depth_stats.png')
plt.clf()

In this case, it appears that the decision tree starts over-fitting at deeper tree levels. 
An optimal tree depth appears to be around ten on this dataset.

# Maximum Bins
Finally, we will perform our evaluation on the impact of setting the number of bins for the decision tree. 
As with the tree depth, a larger number of bins should allow the model to become more complex and might help performance with larger feature dimensions. 
After a certain point, it is unlikely that it will help any more and might, in fact, hinder performance on the test set due to over-fitting.

In [ ]:
params = [2, 4, 8, 16, 32, 64, 100]
metrics = [evaluate_dt(train_data_dt, test_data_dt, 5, param) for param in params]
print params
print metrics
plt.plot(params, metrics)
fig = plt.gcf()
process_figure(fig, 'bin_stats.png')
plt.clf()

Here, we will show the output and plot to vary the number of bins (while keeping the tree depth at the default level of five). 
In this case, using a small number of bins hurts performance, while there is no impact when we use around thirty-two bins (the default setting) or more. 
There seems to be an optimal setting for test set performance at around 16-20 bins.

# Summary
In this chapter, you saw how to use MLlib's linear model and decision tree functionality in Python within the context of regression models. 
We explored categorical feature extraction and the impact of applying transformations to the target variable in a regression problem. 
Finally, we implemented various performance-evaluation metrics and used them to implement a cross-validation exercise that explores the impact of the various parameter settings available in both linear models and decision trees on test set model performance.